In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from datetime import time
import matplotlib.pyplot as pplot
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import math
import random
from sklearn.metrics import mean_absolute_error

/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [65]:
v_train = pd.read_csv('treino_agregado.csv')
v_test = pd.read_csv('teste_agregado.csv')

In [66]:
#Adicionando valor da janela de tempo anterior na janela de tempo atual
v_train["volume_anterior"] = v_train.groupby(['direction', 'tollgate_id'])["volume"].transform("shift")
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
v_train["volume_anterior"] =v_train.groupby(['direction', 'tollgate_id'])["volume_anterior"].fillna(method="ffill").fillna(0)

In [67]:
#Adicionando valor da janela de tempo anterior na janela de tempo atual
v_train["volume_anterior_2"] = v_train.groupby(['direction', 'tollgate_id'])["volume"].transform("shift", 2)
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
v_train["volume_anterior_2"] =v_train.groupby(['direction', 'tollgate_id'])["volume_anterior_2"].fillna(method="ffill").fillna(0)

In [69]:
#Adicionando valor da janela de tempo anterior na janela de tempo atual
v_train["volume_proximo"] = v_train.groupby(['direction', 'tollgate_id'])["volume"].transform("shift", -1)
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
v_train["volume_proximo"] =v_train.groupby(['direction', 'tollgate_id'])["volume_proximo"].fillna(method="ffill").fillna(0)

In [71]:
#Adicionando valor da janela de tempo anterior na janela de tempo atual
v_train["volume_proximo_2"] = v_train.groupby(['direction', 'tollgate_id'])["volume"].transform("shift", -2)
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
v_train["volume_proximo_2"] =v_train.groupby(['direction', 'tollgate_id'])["volume_proximo_2"].fillna(method="ffill").fillna(0)

In [73]:
v_train['avg_vol_dia_semana'] = v_train.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean)

In [75]:
v_train['desvio_padrao'] = v_train.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.std)

In [77]:
v_train['desvio_padrao'].fillna(v_train.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)


In [78]:
v_train

,time,tollgate_id,direction,time_window,volume,day,hour,minute,week,weekend,am_pm,volume_anterior,volume_anterior_2,volume_proximo,volume_proximo_2,avg_vol_dia_semana,desvio_padrao
0,2016-09-19 00:00:00,1,0,"[00:00:00,00:20:00)",13,19,0,0,0,0,1,0.0,0.0,6.0,9.0,13.166667,4.750439
1,2016-09-19 00:00:00,1,1,"[00:00:00,00:20:00)",140,19,0,0,0,0,1,0.0,0.0,56.0,31.0,109.833333,45.327328
2,2016-09-19 00:00:00,2,0,"[00:00:00,00:20:00)",2,19,0,0,0,0,1,0.0,0.0,1.0,2.0,2.750000,0.957427
3,2016-09-19 00:00:00,3,0,"[00:00:00,00:20:00)",17,19,0,0,0,0,1,0.0,0.0,16.0,16.0,17.000000,2.756810
4,2016-09-19 00:00:00,3,1,"[00:00:00,00:20:00)",181,19,0,0,0,0,1,0.0,0.0,112.0,93.0,126.000000,52.680167
5,2016-09-19 00:20:00,1,0,"[00:20:00,00:40:00)",6,19,0,20,0,0,1,13.0,0.0,9.0,10.0,10.166667,3.816630
6,2016-09-19 00:20:00,1,1,"[00:20:00,00:40:00)",56,19,0,20,0,0,1,140.0,0.0,31.0,4.0,51.500000,16.428634
7,2016-09-19 00:20:00,3,0,"[00:20:00,00:40:00)",16,19,0,20,0,0,1,17.0,0.0,16.0,17.0,17.166667,3.430258
8,2016-09-19 00:20:00,3,1,"[00:20:00,00:40:00)",112,19,0,20,0,0,1,181.0,0.0,93.0,4.0,83.666667,32.903900
9,2016-09-19 00:40:00,1,0,"[00:40:00,01:00:00)",9,19,0,40,0,0,1,6.0,13.0,10.0,14.0,8.500000,2.664583


In [79]:
v_train.to_csv('data_process_final/treino_final.csv', index=False)

In [ ]:
#AGORA FAZER O MESMO COM OS DADOS DE TESTE

In [80]:
#Adicionando valor da janela de tempo anterior na janela de tempo atual
v_test["volume_anterior"] = v_test.groupby(['direction', 'tollgate_id'])["volume"].transform("shift")
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
v_test["volume_anterior"] =v_test.groupby(['direction', 'tollgate_id'])["volume_anterior"].fillna(method="ffill").fillna(0)

In [81]:
#Adicionando valor da janela de tempo anterior na janela de tempo atual
v_test["volume_anterior_2"] = v_test.groupby(['direction', 'tollgate_id'])["volume"].transform("shift", 2)
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
v_test["volume_anterior_2"] =v_test.groupby(['direction', 'tollgate_id'])["volume_anterior_2"].fillna(method="ffill").fillna(0)

In [82]:
#Adicionando valor da janela de tempo anterior na janela de tempo atual
v_test["volume_proximo"] = v_test.groupby(['direction', 'tollgate_id'])["volume"].transform("shift", -1)
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
v_test["volume_proximo"] =v_test.groupby(['direction', 'tollgate_id'])["volume_proximo"].fillna(method="ffill").fillna(0)

In [83]:
#Adicionando valor da janela de tempo anterior na janela de tempo atual
v_test["volume_proximo_2"] = v_test.groupby(['direction', 'tollgate_id'])["volume"].transform("shift", -2)
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
v_test["volume_proximo_2"] =v_test.groupby(['direction', 'tollgate_id'])["volume_proximo_2"].fillna(method="ffill").fillna(0)

In [85]:
v_test['avg_vol_dia_semana'] = v_test.groupby(['time_window', 'direction', 'tollgate_id'])["volume"].transform(np.mean)

In [86]:
v_test['desvio_padrao'] = v_test.groupby(['time_window', 'direction', 'tollgate_id'])["volume"].transform(np.std)

In [87]:
v_test['desvio_padrao'].fillna(v_test.groupby(['time_window', 'week', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)


In [88]:
v_test

,time,tollgate_id,direction,time_window,volume,day,hour,minute,week,weekend,am_pm,volume_anterior,volume_anterior_2,volume_proximo,volume_proximo_2,avg_vol_dia_semana,desvio_padrao
0,2016-10-25 06:00:00,1,0,"[06:00:00,06:20:00)",10,25,6,0,1,0,1,0.0,0.0,17.0,30.0,14.285714,6.447591
1,2016-10-25 06:00:00,1,1,"[06:00:00,06:20:00)",37,25,6,0,1,0,1,0.0,0.0,54.0,63.0,36.571429,2.992053
2,2016-10-25 06:00:00,2,0,"[06:00:00,06:20:00)",29,25,6,0,1,0,1,0.0,0.0,27.0,40.0,19.714286,9.655988
3,2016-10-25 06:00:00,3,0,"[06:00:00,06:20:00)",28,25,6,0,1,0,1,0.0,0.0,48.0,92.0,24.142857,4.298394
4,2016-10-25 06:00:00,3,1,"[06:00:00,06:20:00)",30,25,6,0,1,0,1,0.0,0.0,31.0,42.0,21.428571,5.126960
5,2016-10-25 06:20:00,1,0,"[06:20:00,06:40:00)",17,25,6,20,1,0,1,10.0,0.0,30.0,25.0,22.428571,14.808299
6,2016-10-25 06:20:00,1,1,"[06:20:00,06:40:00)",54,25,6,20,1,0,1,37.0,0.0,63.0,67.0,47.714286,6.524678
7,2016-10-25 06:20:00,2,0,"[06:20:00,06:40:00)",27,25,6,20,1,0,1,29.0,0.0,40.0,83.0,24.000000,13.686977
8,2016-10-25 06:20:00,3,0,"[06:20:00,06:40:00)",48,25,6,20,1,0,1,28.0,0.0,92.0,129.0,45.428571,6.876461
9,2016-10-25 06:20:00,3,1,"[06:20:00,06:40:00)",31,25,6,20,1,0,1,30.0,0.0,42.0,62.0,28.857143,5.698789


In [89]:
v_test.to_csv('data_process_final/teste_final.csv', index=False)

In [91]:
df_train = pd.read_csv('data_process_final/treino_final.csv')
df_teste = pd.read_csv('data_process_final/teste_final.csv')

In [94]:
def feature_format():
    #pd_volume_train = pd_volume_train.set_index(['time'])
    #pd_volume_test = pd_volume_test.set_index(['time'])
    #volume_train = v_train.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #volume_test = v_test.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #print(volume_train)                
    x = pd.Series(df_train['time_window'].unique())
    s = pd.Series(range(len(x)),index = x.values)
    df_train['window_n'] = df_train['time_window'].map(s)
    df_teste['window_n'] = df_teste['time_window'].map(s)
#        print vol_test.tail()
    #volume_train['weekday'] = v_train['weekday']
    #volume_test['weekday'] = v_test['weekday']
    
    feature_train = df_train.drop('volume', axis = 1)
    feature_test = df_teste.drop('volume',axis = 1)
    values_train = df_train['volume'].values
    values_test = df_teste['volume'].values
    
    return feature_train, feature_test, values_train, values_test

In [95]:
feature_train, feature_test, values_train, values_test = feature_format()

In [96]:
feature_train.head()

,time,tollgate_id,direction,time_window,day,hour,minute,week,weekend,am_pm,volume_anterior,volume_anterior_2,volume_proximo,volume_proximo_2,avg_vol_dia_semana,desvio_padrao,window_n
0,2016-09-19 00:00:00,1,0,"[00:00:00,00:20:00)",19,0,0,0,0,1,0.0,0.0,6.0,9.0,13.166667,4.750439,0
1,2016-09-19 00:00:00,1,1,"[00:00:00,00:20:00)",19,0,0,0,0,1,0.0,0.0,56.0,31.0,109.833333,45.327328,0
2,2016-09-19 00:00:00,2,0,"[00:00:00,00:20:00)",19,0,0,0,0,1,0.0,0.0,1.0,2.0,2.750000,0.957427,0
3,2016-09-19 00:00:00,3,0,"[00:00:00,00:20:00)",19,0,0,0,0,1,0.0,0.0,16.0,16.0,17.000000,2.756810,0
4,2016-09-19 00:00:00,3,1,"[00:00:00,00:20:00)",19,0,0,0,0,1,0.0,0.0,112.0,93.0,126.000000,52.680167,0


In [97]:
rng = np.random.RandomState(1)
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 50),
                         n_estimators=300, random_state = rng)

In [119]:
regr.fit(feature_train[['tollgate_id', 'direction', 'week', 'weekend', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao']], values_train)

y_pred = regr.predict(feature_test[['tollgate_id', 'direction', 'week', 'weekend', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao']])

In [99]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

In [120]:
mean_absolute_percentage_error(values_test, y_pred)

0.24156422297913924

In [122]:
rf = RandomForestRegressor(n_estimators = 1500, random_state = 42)

In [123]:
rf.fit(feature_train[['tollgate_id', 'direction', 'week', 'weekend', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao']], values_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1500, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [124]:
# Use the forest's predict method on the test data
predictions = rf.predict(feature_test[['tollgate_id', 'direction', 'week', 'weekend', 'am_pm', 'volume_anterior', 'volume_anterior_2', 'volume_proximo', 'volume_proximo_2', 'avg_vol_dia_semana', 'desvio_padrao']])
# Calculate the absolute errors
errors = abs(predictions - values_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 11.6 degrees.


In [125]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / values_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 74.8 %.


In [126]:
mean_absolute_percentage_error(values_test, predictions)

0.25197467203766294